# Segmentation en phrases

## Imports

In [32]:
import os
import sys
import nltk
from nltk.tokenize import sent_tokenize

In [33]:
import sys
import nltk
from nltk.tokenize import sent_tokenize

## Fichiers d'inputs et d'outputs

In [34]:
infile = "../tp4/corpus.txt"
outfile = "../tp4/corpusw3v.txt"

## Segmentation en phrases du corpus complet et création d'un nouveau fichier

**Important** : pour traiter le corpus complet, indiquez `LIMIT = None`

In [35]:
LIMIT = None

In [36]:
with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        content = f.readlines()
        content = content[:LIMIT] if LIMIT is not None else content
        n_lines = len(content)
        for i, line in enumerate(content):
            if i % 100 == 0:
                print(f'processing line {i}/{n_lines}')
            sentences = sent_tokenize(line)
            for sent in sentences:
                output.write(sent + "\n")
print("Done")

processing line 0/1


# Word Embeddings : le modèle Word2Vec

## Imports

In [6]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [13]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [14]:
infile = f"../tp4/corpusw3v.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [22]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [23]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [24]:
len(bigram_phrases.vocab.keys())

13333

Prenons une clé au hasard :

In [25]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

u_b


Le dictionnaire indique le score de cette coocurrence :

In [26]:
bigram_phrases.vocab[key_]

5015

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [27]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [28]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [29]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [30]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [31]:
print(corpus[:100])

['DEUXIEME ANNEE.', '- N. 130.', '20 centimes le numéro i - liflERCRfcDI 3 JUIN 1925, DRAPEAU téléphone .', ': Administration-Publicité . . ... . . ... ...., 16333 * Rédaction 163.33 » Rédaction {après 19 heures) 296.89 Adressa télégraphique : DRAPOUGE-BRUXELLEÔ Compte-chèques postaux ; n" 122.595 Organe quotidien du Parti Communiste Beïge (S B I 57-5S Rue dea A»eaens, 57-59 BkO/ŒLLEL> *>, ABONNEMENTS : Belgique, 30 septembre 31 décent > Etranger.', '1 an .', "i t'abonne dans tous tes, bureaux de punis.", "Les dettes belges envers les Etats-Unis Vers l'inflation et la main-mise américaine sur l'industrie belge Grâce aux formidables emprunts contractés aux Etats-Unis d'Amérique du- rant la guerre capitaliste de 1911 ct pondant les années suivant l'armistice, les banquiers américains .ont étendu la puissance de leur contrôle sur une grande partie des pays bourgeois d'Europe.", "L'expansion impérialiste des Etats- Unis s'opère silencieusement et* pacifiquement grâce au dollar.", "Le gouve

## Entrainement d'un modèle Word2Vec sur ce corpus

In [22]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=7, # On ignore les mots qui n'apparaissent pas au moins 7 fois dans le corpus
    workers=3, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 3.55 s
Wall time: 6.47 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [24]:
outfile = f"../tp4/corpusdb.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [25]:
model = Word2Vec.load("../tp4/corpusdb.model")

### Imprimer le vecteur d'un terme

In [27]:
model.wv["banque"]

array([-0.13188373, -0.6026362 ,  0.33602905,  2.333313  ,  0.21708763,
        0.43139032,  0.04926314,  1.820423  ,  0.67257786,  1.4755898 ,
       -0.44795865, -2.3396027 ,  0.54695386, -0.37319398, -1.8555108 ,
        0.38626394, -1.310726  ,  0.7865423 , -0.6655431 ,  2.9735935 ,
       -0.49011585, -0.00877105,  1.8593655 , -3.4466689 ,  3.039669  ,
       -0.3737799 ,  1.617716  , -1.4171431 , -1.0412405 , -1.4924067 ,
       -0.01603176, -0.4810704 ], dtype=float32)

In [28]:
model.wv["etat"]

array([ 0.4951036 , -0.23177837, -1.0529529 ,  0.37591457, -0.24939391,
       -1.6867355 , -0.26483667,  0.30665636, -0.79198337,  1.8225222 ,
        0.32028905, -0.84671676,  0.20436318, -0.02616134, -0.00405181,
       -0.05192585,  0.12048884,  0.400746  ,  1.1258572 ,  0.17232727,
        0.7332141 ,  1.3255436 ,  1.4142447 , -0.5056767 ,  0.20474449,
       -0.3070956 , -0.58283293, -0.8261282 ,  0.2706182 ,  0.43015268,
       -0.05444039,  0.28174654], dtype=float32)

### Calculer la similarité entre deux termes

In [54]:
model.wv.similarity("cours", "obligations")

0.50232005

In [57]:
model.wv.similarity("obligations", "obligation")

0.6350837

In [60]:
model.wv.similarity("france", "belgique")



0.84518313

### Chercher les mots les plus proches d'un terme donné

In [61]:
model.wv.most_similar("noir", topn=5)

[('bleu', 0.9553493857383728),
 ('rose', 0.9520599842071533),
 ('jaune', 0.9505562782287598),
 ('transvaal', 0.9497373700141907),
 ('renard', 0.9476518630981445)]

In [62]:
model.wv.most_similar("soir", topn=5)

[('matin', 0.9528449773788452),
 ('siege', 0.9342163801193237),
 ('drame', 0.9251589179039001),
 ('courrier', 0.9147465229034424),
 ('roi', 0.9114739894866943)]

In [63]:
model.wv.most_similar("actions", topn=5)

[('jouissance', 0.842002809047699),
 ('actions_privilegiees', 0.840703010559082),
 ('depots', 0.8149512410163879),
 ('obligations', 0.8100811839103699),
 ('liberees', 0.806624174118042)]

In [64]:
model.wv.most_similar("reine", topn=5)

[('coupable', 0.956748902797699),
 ('veuve', 0.9566851854324341),
 ('la_danse', 0.9558706879615784),
 ('la_princesse', 0.9555102586746216),
 ('pare', 0.9547945857048035)]

In [65]:
model.wv.most_similar("homme", topn=5)

[('le_roi', 0.9521457552909851),
 ('seul', 0.9495663046836853),
 ('assure', 0.9463509321212769),
 ('aussitot', 0.9461931586265564),
 ('grave', 0.9457772374153137)]

In [68]:
model.wv.most_similar("fer", topn=5)

[('chemin', 0.7993881702423096),
 ('du_congo', 0.7872655391693115),
 ('des_chemins', 0.778247058391571),
 ('du_chemin', 0.7752842307090759),
 ('charleroi', 0.7616026401519775)]

In [77]:
model.wv.most_similar("esclaves", topn=5)

[('plus_belles', 0.9268314242362976),
 ('moindres', 0.9179300665855408),
 ('cours_actuels', 0.9111868143081665),
 ('yeux', 0.9066286087036133),
 ('incidents', 0.9065471887588501)]

### Faire des recherches complexes à travers l'espace vectoriel

In [78]:
print(model.wv.most_similar(positive=['actions', 'titres'], negative=['belge']))

[('nouvelles', 0.8084534406661987), ('ont_ete', 0.7788301110267639), ('sieges', 0.7756906747817993), ('livres', 0.7710818648338318), ('cinq', 0.7646100521087646), ('quatre', 0.762945830821991), ('diverses', 0.7597044110298157), ('divers', 0.7593969106674194), ('au_lieu', 0.7544440031051636), ('effets', 0.7464857697486877)]


In [79]:
print(model.wv.most_similar(positive=['congo', 'affaires'], negative=['belge']))

[('distingues', 0.7993931174278259), ('mites', 0.7957741022109985), ('remarquables', 0.7790096402168274), ('mines_aljustrel', 0.771826446056366), ('chauffeurs', 0.7674798965454102), ('revolutionnaires', 0.7655134797096252), ('sont_soutenues', 0.7652339935302734), ('esclaves', 0.7636707425117493), ('dieux', 0.7617565393447876), ('parmi_les_exceptions', 0.758766770362854)]


In [50]:
print(model.wv.most_similar(positive=['noir', 'blanc'], negative=['couleur']))

[('oct', 0.854617714881897), ('danube', 0.8488488793373108), ('mara', 0.8435136675834656), ('plau', 0.830959677696228), ('cmb', 0.8236958384513855), ('emb', 0.8222048878669739), ('livr', 0.8221510648727417), ('bombay', 0.8182555437088013), ('lin_am', 0.8067922592163086), ('allem', 0.8059899806976318)]


In [52]:
print(model.wv.most_similar(positive=['banquier', 'metier'], negative=['communication']))

[('comte', 0.8907211422920227), ('cliateau', 0.8722466230392456), ('bralne', 0.8714914917945862), ('trio', 0.8680735230445862), ('poussin', 0.858370840549469), ('sacre', 0.8534082770347595), ('mot', 0.8528465628623962), ('sourire', 0.851472020149231), ('drole', 0.8499080538749695), ('ajouta', 0.8454453349113464)]
